In [295]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload
import os
from tsetlin import TsetlinMachine
import torch
import random

DATASET_DIR = '../datasets/'
DATA_FILE = 'bit_1.txt'
SEED = None

text_rows = open(f'{DATASET_DIR}{DATA_FILE}', 'r').read().splitlines()
dataset = [ [int(num) for num in row.split(',')] for row in text_rows]
tensor_dataset = torch.tensor(dataset)
train_x = tensor_dataset[:, :-1]
train_y = tensor_dataset[:, -1]

if SEED:
    random.seed(SEED)
    torch.manual_seed(SEED)
else:
    seed = int.from_bytes(os.urandom(8), byteorder="big", signed=False)
    random.seed(seed)
    torch.manual_seed(seed)
    print(seed)

tm = TsetlinMachine(train_x.shape[1], 5)

out_1 = tm.forward(train_x)
tm.update(train_y)
out_2 = tm.forward(train_x)
print(out_2, train_y)
assert torch.equal(train_y,out_2)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
15305054968550633065
tensor([1, 1, 1, 1, 0, 0, 0, 0]) tensor([1, 1, 1, 1, 0, 0, 0, 0])


In [296]:
tm.l1.W, tm.l2.W, tm.l3.W

(tensor([[0, 0, 0, 1, 0, 0],
         [0, 0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0, 0]]),
 tensor([[0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
         [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]),
 tensor([[1, 0, 0, 0, 0, 0, 0, 0, 0, 0]]))

In [297]:
for _ in range(100):
    seed = int.from_bytes(os.urandom(8), byteorder="big", signed=False)
    random.seed(seed)
    torch.manual_seed(seed)
    # print(seed)
    
    shuffled_idx = torch.randperm(train_x.shape[0])
    shuffled_x = train_x[shuffled_idx]
    shuffled_y = train_y[shuffled_idx]
    
    tm = TsetlinMachine(train_x.shape[1], 5)
    out_1 = tm.forward(shuffled_x)
    tm.update(shuffled_y)
    out_2 = tm.forward(shuffled_x)
    assert torch.equal(shuffled_y,out_2)

In [298]:
SEED = 14756477415146734378

if SEED:
    random.seed(SEED)
    torch.manual_seed(SEED)
else:
    seed = int.from_bytes(os.urandom(8), byteorder="big", signed=False)
    random.seed(seed)
    torch.manual_seed(seed)
    print(seed)

shuffled_idx = torch.randperm(train_x.shape[0])
shuffled_x = train_x[shuffled_idx]
shuffled_y = train_y[shuffled_idx]
tm = TsetlinMachine(train_x.shape[1], 5)
for i in range(15):
    batch_idxs = torch.randint(0, shuffled_x.shape[0], (2,))
    batch_x = shuffled_x[batch_idxs]
    batch_y = shuffled_y[batch_idxs]
    out_1 = tm.forward(batch_x)
    tm.update(batch_y)
    out_2 = tm.forward(batch_x)
    assert torch.equal(batch_y,out_2)

out_3 = tm.forward(train_x)
assert torch.equal(train_y,out_3)

In [404]:
failed_training = 0
for _ in range(1000):
    SEED = None
    if SEED:
        random.seed(SEED)
        torch.manual_seed(SEED)
    else:
        seed = int.from_bytes(os.urandom(8), byteorder="big", signed=False)
        random.seed(seed)
        torch.manual_seed(seed)
        # print(seed)

    shuffled_idx = torch.randperm(train_x.shape[0])
    shuffled_x = train_x[shuffled_idx]
    shuffled_y = train_y[shuffled_idx]
    tm = TsetlinMachine(train_x.shape[1], 5)
    for i in range(100):
        batch_idxs = torch.randint(0, shuffled_x.shape[0], (2,))
        batch_x = shuffled_x[batch_idxs]
        batch_y = shuffled_y[batch_idxs]
        out_1 = tm.forward(batch_x)
        tm.update(batch_y)
        out_2 = tm.forward(batch_x)
        assert torch.equal(batch_y,out_2)

    out_3 = tm.forward(train_x)
    if not torch.equal(train_y,out_3):
        failed_training += 1

failed_training

110

In [408]:
failed_training = 0
failed_tms = []
succeeded_tms = []
for _ in range(1000):
    SEED = None
    if SEED:
        random.seed(SEED)
        torch.manual_seed(SEED)
    else:
        seed = int.from_bytes(os.urandom(8), byteorder="big", signed=False)
        random.seed(seed)
        torch.manual_seed(seed)

    shuffled_idx = torch.randperm(train_x.shape[0])
    shuffled_x = train_x[shuffled_idx]
    shuffled_y = train_y[shuffled_idx]
    tm = TsetlinMachine(train_x.shape[1], 5)
    for i in range(100):
        batch_idxs = torch.randint(0, shuffled_x.shape[0], (1,))
        batch_x = shuffled_x[batch_idxs]
        batch_y = shuffled_y[batch_idxs]
        out_1 = tm.forward(batch_x)
        tm.update(batch_y)
        out_2 = tm.forward(batch_x)
        assert torch.equal(batch_y,out_2)

    out_3 = tm.forward(train_x)
    if not torch.equal(train_y,out_3):
        failed_training += 1
        failed_tms.append(tm)
    else:
        succeeded_tms.append(tm)

failed_training

0

In [393]:
SEED = 9567674845606600831
if SEED:
    random.seed(SEED)
    torch.manual_seed(SEED)
else:
    seed = int.from_bytes(os.urandom(8), byteorder="big", signed=False)
    random.seed(seed)
    torch.manual_seed(seed)
    print(seed)

shuffled_idx = torch.randperm(train_x.shape[0])
shuffled_x = train_x[shuffled_idx]
shuffled_y = train_y[shuffled_idx]
tm = TsetlinMachine(train_x.shape[1], 5)
for i in range(100):
    batch_idxs = torch.randint(0, shuffled_x.shape[0], (1,))
    batch_x = shuffled_x[batch_idxs]
    batch_y = shuffled_y[batch_idxs]
    out_1 = tm.forward(batch_x)
    tm.update(batch_y)
    out_2 = tm.forward(batch_x)
    assert torch.equal(batch_y,out_2)

out_3 = tm.forward(train_x)
if not torch.equal(train_y,out_3):
    print("FAILED")